# WRDS CRSP Documentation

### Treasury Libraries
`crsp_m_treasuries`

Quote Data
* tfz_dly: main file of treasury quotes
* tfz_dly_ts2: supplemental series
* tfz_dly_ft: matched to term structure at any given date
* tfz_dly_cd: compilation of fed reported rates in tfz format
* tfz_dly_cpi: compilation of cpi data
* tfz_dly_rf2: compiled risk-free rate

Identifying Info
* tfz_iss: issue details (which are included automatically in queries.)
* tfz_idx: description of KYTREASNOX codes

### WRDS Python Manual
https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/

https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/python-example-data-workflow/

https://wrds-www.wharton.upenn.edu/pages/support/applications/python-replications/fama-french-factors-python/

### CRSP Treasury Manual

https://www.crsp.org/products/documentation/treasno-properties
https://www.crsp.org/products/documentation/reference-information

In [1]:
import wrds
file_key = open("../../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

# Library Info

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')
display(treasury_databases)

['tfz_dly',
 'tfz_dly_cd',
 'tfz_dly_cpi',
 'tfz_dly_ft',
 'tfz_dly_rf2',
 'tfz_dly_ts2',
 'tfz_idx',
 'tfz_iss',
 'tfz_mast',
 'tfz_mth',
 'tfz_mth_bp',
 'tfz_mth_cd',
 'tfz_mth_cpi',
 'tfz_mth_fb',
 'tfz_mth_ft',
 'tfz_mth_rf',
 'tfz_mth_rf2',
 'tfz_mth_ts',
 'tfz_mth_ts2',
 'tfz_pay']

In [4]:
db.describe_table(library='crsp_m_treasuries', table='tfz_dly')

Approximately 3266406 rows in crsp_m_treasuries.tfz_dly.


,name,nullable,type,comment
0,kytreasno,True,DOUBLE_PRECISION,None
1,kycrspid,True,VARCHAR(15),None
2,caldt,True,DATE,None
3,tdbid,True,DOUBLE_PRECISION,None
4,tdask,True,DOUBLE_PRECISION,None
5,tdnomprc,True,DOUBLE_PRECISION,None
6,tdnomprc_flg,True,VARCHAR(1),None
7,tdsourcr,True,VARCHAR(1),None
8,tdaccint,True,DOUBLE_PRECISION,None
9,tdretnua,True,DOUBLE_PRECISION,None


# Database quote

In [15]:
DATE = '2022-09-30'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg


In [12]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204079.0,20220815.107250,20220815.107250,912810EM,1992-08-15,2022-08-15,1.0,7.250,2.0,3.625000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204080.0,20221115.107620,20221115.107620,912810EN,1992-11-15,2022-11-15,1.0,7.625,2.0,3.812500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204081.0,20230215.107120,20230215.107120,912810EP,1993-02-15,2023-02-15,0.0,7.125,2.0,3.562500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204082.0,20230815.106250,20230815.106250,912810EQ,1993-08-16,2023-08-15,0.0,6.250,2.0,3.108016,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204083.0,20241115.107500,20241115.107500,912810ES,1994-05-15,2024-11-15,0.0,7.500,2.0,3.750000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,207818.0,20221229.400000,20221229.400000,912796R2,2021-12-30,2022-12-29,0.0,0.000,0.0,0.000000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
418,207819.0,20231231.200750,20231231.200750,91282CDR,2021-12-31,2023-12-31,0.0,0.750,2.0,0.375000,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
419,207820.0,20241215.201000,20241215.201000,91282CDN,2021-12-15,2024-12-15,0.0,1.000,2.0,0.500000,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
420,207821.0,20261231.201250,20261231.201250,91282CDQ,2021-12-31,2026-12-31,0.0,1.250,2.0,0.625000,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


In [13]:
data = data_quotes.merge(data_iss, on='kytreasno')
data

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204082.0,20230815.106250,2021-12-31,109.101562,109.125000,109.113281,M,I,2.343750,-0.000408,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204080.0,20221115.107620,2021-12-31,106.296875,106.320312,106.308594,M,I,0.968923,-0.000422,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204079.0,20220815.107250,2021-12-31,104.257812,104.281250,104.269531,M,I,2.718750,-0.000400,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204081.0,20230215.107120,2021-12-31,107.406250,107.429688,107.417969,M,I,2.671875,-0.000321,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204092.0,20280815.105500,2021-12-31,125.796875,125.843750,125.820312,M,I,2.062500,-0.000066,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,207812.0,20220426.400000,2021-12-31,99.969389,99.972611,99.971000,M,I,0.000000,0.000019,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
418,207819.0,20231231.200750,2021-12-31,100.011719,100.050781,100.031250,M,I,0.000000,-0.000156,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
419,207822.0,20281231.201370,2021-12-31,99.554688,99.601562,99.578125,M,I,0.000000,-0.000275,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
420,207816.0,20220630.400000,2021-12-31,99.904472,99.909500,99.906986,M,I,0.000000,-0.000020,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


# Get Yield Curve Data (CRSP)

In [8]:
data_ft = db.get_table(library='crsp_m_treasuries', table='tfz_dly_ft').pivot_table(index='caldt',values='tdytm', columns='kytreasnox')
data_ft.columns = [1,2,5,7,10,20,30]
data_ft

,1,2,5,7,10,20,30
caldt,,,,,,,
1961-06-14,2.935907,3.360687,3.623677,3.768720,3.818819,3.814210,3.815172
1961-06-15,2.932936,3.376460,3.671691,3.804225,3.862987,3.828220,3.826316
1961-06-16,2.929949,3.375670,3.685431,3.804216,3.863282,3.832922,3.830049
1961-06-19,2.920884,3.389970,3.712984,3.824557,3.886205,3.842378,3.837543
1961-06-20,2.952419,3.355796,3.685391,3.809274,3.886506,3.856465,3.845018
...,...,...,...,...,...,...,...
2022-11-23,4.666103,4.465921,3.886053,3.657613,3.665568,4.010443,3.723134
2022-11-25,4.654875,4.442006,3.859520,3.629335,3.652183,3.995460,3.735102
2022-11-28,4.635645,4.420958,3.868822,3.640823,3.650411,3.996127,3.720517


# Get Timeseries for certain maturities

In [24]:
MATDT = '2022-02-15'

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where tmatdt=\'{MATDT}\''
temp = db.raw_sql(SQL_QUERY)
#temp = temp[temp['itype'].isin([11,12]) == DOTIPS]
temp

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,206339.0,20220215.202000,20220215.202000,912828SF,2012-02-15,2022-02-15,1.0,2.0,2.0,1.00,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
1,207309.0,20220215.202500,20220215.202500,9128286C,2019-02-15,2022-02-15,1.0,2.5,2.0,1.25,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
2,207781.0,20220215.400000,20220215.400000,912796R8,2021-10-19,2022-02-15,1.0,0.0,0.0,0.00,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


In [25]:
if temp.shape[0]>1:
    KEYNO = tuple(temp['kytreasno'])
    SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO}'
else:
    KEYNO = temp['kytreasno'].values[0]
    SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno =\'{KEYNO}\''
    
ts = db.raw_sql(SQL_QUERY)
ts

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,206339.0,20220215.202000,2012-02-09,99.640625,99.703125,99.671875,M,I,0.0,NaN,0.000055,3334.278135,NaN,NaN,0.0,None,None
1,206339.0,20220215.202000,2012-02-10,100.093750,100.140625,100.117188,M,I,0.0,0.004468,0.000054,3334.172369,NaN,NaN,0.0,None,None
2,206339.0,20220215.202000,2012-02-13,100.187500,100.250000,100.218750,M,I,0.0,0.001014,0.000054,3331.345227,NaN,NaN,0.0,None,None
3,206339.0,20220215.202000,2012-02-14,100.546875,100.593750,100.570312,M,I,0.0,0.003508,0.000053,3331.044132,NaN,NaN,0.0,None,None
4,206339.0,20220215.202000,2012-02-15,100.625000,100.687500,100.656250,M,I,0.0,0.000855,0.000053,3330.206381,NaN,NaN,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,207781.0,20220215.400000,2022-01-25,99.997083,99.997667,99.997375,M,I,0.0,0.000007,0.000001,21.000000,NaN,61711.0,0.0,None,None
3347,207781.0,20220215.400000,2022-01-26,99.997222,99.997778,99.997500,M,I,0.0,0.000001,0.000001,20.000000,NaN,61711.0,0.0,None,None
3348,207781.0,20220215.400000,2022-01-27,99.997361,99.997889,99.997625,M,I,0.0,0.000001,0.000001,19.000000,NaN,61711.0,0.0,None,None
3349,207781.0,20220215.400000,2022-01-28,99.997750,99.998250,99.998000,M,I,0.0,0.000004,0.000001,18.000000,NaN,61711.0,0.0,None,None


# Get Timeseries for certain issues

In [15]:
KEYNO_ts = (207816,207818)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_ts}'

ts_issue = db.raw_sql(SQL_QUERY)
ts_issue

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,207818.0,20221229.400000,2022-01-12,99.561250,99.571000,99.566125,M,I,0.0,-0.000110,0.000012,351.0,None,38798.0,0.0,None,None
1,207816.0,20220630.400000,2022-03-23,99.862500,99.865250,99.863875,M,I,0.0,0.000097,0.000014,99.0,None,58198.0,0.0,None,None
2,207816.0,20220630.400000,2022-03-24,99.861167,99.863889,99.862528,M,I,0.0,-0.000013,0.000014,98.0,None,58198.0,0.0,None,None
3,207816.0,20220630.400000,2022-03-25,99.846417,99.859889,99.853153,M,I,0.0,-0.000094,0.000015,97.0,None,58198.0,0.0,None,None
4,207816.0,20220630.400000,2022-03-28,99.848556,99.853778,99.851167,M,I,0.0,-0.000020,0.000016,94.0,None,58198.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,207818.0,20221229.400000,2022-03-25,99.062250,99.070000,99.066125,M,I,0.0,-0.001240,0.000034,279.0,None,38798.0,0.0,None,None
212,207818.0,20221229.400000,2022-03-28,98.988000,99.041667,99.014833,M,I,0.0,-0.000518,0.000036,276.0,None,38798.0,0.0,None,None
213,207818.0,20221229.400000,2022-03-29,99.018403,99.083333,99.050868,M,I,0.0,0.000364,0.000035,275.0,None,38798.0,0.0,None,None
214,207818.0,20221229.400000,2022-03-30,99.079056,99.086667,99.082861,M,I,0.0,0.000323,0.000034,274.0,None,38798.0,0.0,None,None


# Get Timeseries for TIPS issues

In [16]:
KEYNO_tips = (207431,206999,207009)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_tips}'

ts_issue_tips = db.raw_sql(SQL_QUERY)
ts_issue_tips

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,207431.0,20241015.C00120,2019-10-10,99.765625,99.796875,99.781250,M,I,0.0,NaN,NaN,NaN,NaN,NaN,0.0,1.00001,C
1,206999.0,20260715.C00120,2016-12-23,95.968750,96.101562,96.035156,M,I,0.0,NaN,NaN,NaN,23999.0,37103.0,0.0,1.00809,C
2,206999.0,20260715.C00120,2016-12-27,95.917969,96.050781,95.984375,M,I,0.0,NaN,NaN,NaN,23999.0,37103.0,0.0,1.00826,C
3,206999.0,20260715.C00120,2016-12-28,96.121094,96.253906,96.187500,M,I,0.0,NaN,NaN,NaN,23999.0,37103.0,0.0,1.00830,C
4,206999.0,20260715.C00120,2016-12-29,96.214844,96.347656,96.281250,M,I,0.0,NaN,NaN,NaN,23999.0,37103.0,0.0,1.00834,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,207431.0,20241015.C00120,2022-05-24,102.863281,102.898438,102.880859,M,I,0.0,NaN,NaN,NaN,31724.0,38889.0,0.0,1.11678,C
3581,207431.0,20241015.C00120,2022-05-25,102.882812,102.917969,102.900391,M,I,0.0,NaN,NaN,NaN,31724.0,38889.0,0.0,1.11725,C
3582,207431.0,20241015.C00120,2022-05-26,103.136719,103.171875,103.154297,M,I,0.0,NaN,NaN,NaN,31724.0,38889.0,0.0,1.11773,C
3583,207431.0,20241015.C00120,2022-05-27,103.109375,103.144531,103.126953,M,I,0.0,NaN,NaN,NaN,31724.0,38889.0,0.0,1.11821,C


# Get Timeseries for Duration Analysis

In [13]:
KEYNO_ts_duration = (207392, 207391, 207457)
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_ts_duration}'
ts_issue_duration = db.raw_sql(SQL_QUERY)
ts_issue_duration

infoflds = ['tdatdt','tmatdt','tcouprt','itype']
labels = ['issue date','maturity date','coupon rate','security type']
rename_dict = {infoflds[i]:labels[i] for i in range(len(infoflds))}
info_ts_duration = data_iss.set_index('kytreasno').loc[list(KEYNO_ts_duration),infoflds].rename(columns=rename_dict)

# Save to Excel

In [14]:
outfile = f'../data/treasury_quotes_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=False)

In [28]:
outfile = f'../data/treasury_ts_{MATDT}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts.to_excel(writer, sheet_name= 'ts', index=False)

In [29]:
outfile = f'../data/treasury_ts_issue_{KEYNO_ts}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts_issue.to_excel(writer, sheet_name= 'ts', index=False)

In [34]:
outfile = f'../data/treasury_ts_issue_TIPS_{KEYNO_tips}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts_issue_tips.to_excel(writer, sheet_name= 'ts', index=False)

In [12]:
outfile = f'../data/yields.xlsx'
with pd.ExcelWriter(outfile) as writer:
    data_ft.to_excel(writer, sheet_name= 'yields', index=True)

In [14]:
outfile = f'../data/treasury_ts_issue_duration_{KEYNO_ts_duration}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    info_ts_duration.to_excel(writer, sheet_name= 'info', index=True)
    ts_issue_duration.to_excel(writer, sheet_name= 'ts', index=False)